Often you need to process two raster datasets together to create a new raster output and then save that output as a new file. In this lesson, you will learn how to subtract rasters and create a new GeoTIFF file in open source Python using rioxarray which is a wrapper package that adds additional spatial functions to xarray.


In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import seaborn as sns
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import mapping


# Prettier plotting with seaborn
sns.set(font_scale=1.5, style="whitegrid")

# Get data and set working directory
# Define relative path to file
lidar_dem_path = os.path.join("data","colorado-flood",
                              "spatial",
                              "boulder-leehill-rd",
                              "pre-flood",
                              "lidar",
                              "pre_DTM.tif")

Open and plot the lidar digital elevation model (DEM). Note that when you read the data, you can use the argument ```masked = True``` to ensure that the no data values do not plot and are assigned ```nan``` or ```nodata```.


In [ ]:
# Open lidar dem
lidar_dem_xr = rxr.open_rasterio(lidar_dem_path, masked=True).squeeze()
lidar_dem_xr

### Import Digital Surface Model (DSM)
Next, you will open the digital surface model (DSM). The DSM represents the top of the earth’s surface. Thus, it includes trees, buildings and other objects that sit on the earth.

In [ ]:
# Define relative path to file
lidar_dsm_path = os.path.join("data","colorado-flood",
                              "spatial",
                              "boulder-leehill-rd",
                              "pre-flood",
                              "lidar",
                              "pre_DSM.tif")

# Open lidar dem
lidar_dsm_xr = rxr.open_rasterio(lidar_dsm_path, masked=True).squeeze()
lidar_dsm_xr

### Canopy Height Model

The canopy height model (CHM) represents the HEIGHT of the trees. This is not an elevation value, rather it’s the height or distance between the ground and the top of the trees (or buildings or whatever object that the lidar system detected and recorded).

Some canopy height models also include buildings, so you need to look closely at your data to make sure it was properly cleaned before assuming it represents all trees!

Calculate difference between two rasters
There are different ways to calculate a CHM. One easy way is to subtract the DEM from the DSM.

**DSM - DEM = CHM**

This math gives you the residual value or difference between the top of the earth surface and the ground which should be the heights of the trees (and buildings if the data haven’t been “cleaned”).

Before you subtract the two rasters. Before performing this calculate however you should check to ensure that they cover the same spatial extent and are of the same spatial resolution (pixel size).



In [ ]:
# Are the bounds the same?
print("Is the spatial extent the same?",
      lidar_dem_xr.rio.bounds() == lidar_dsm_xr.rio.bounds())

# Is the resolution the same ??
print("Is the resolution the same?",
      lidar_dem_xr.rio.resolution() == lidar_dsm_xr.rio.resolution())

It looks like the bounds and resolution are the same. This means it is safe for you to subtract the two rasters without significant errors or uncertainty introduced.

Below you calculate the difference between the two arrays to generate a Canopy Height Model. You then plot your newly created canopy height model.

In [ ]:
lidar_chm_xr = lidar_dsm_xr - lidar_dem_xr

# Plot the data
f, ax = plt.subplots(figsize=(10, 5))
lidar_chm_xr.plot(cmap="Greens")
ax.set(title="Canopy Height Model for Lee Hill Road Pre-Flood")
ax.set_axis_off()
plt.show()

Plot a histogram to explore the range of raster values in your newly created CHM data.



In [ ]:
lidar_chm_xr.plot.hist()
plt.show()

Take a close look at the CHM plot. Do you think that the data just represents trees? Or do you see anything that may suggest that there are other types of objects represented in the data?

### Explore the CHM Data
Next, explore the data values in your CHM. Think about the values representing things like trees and buildings in your data.

Do the data make sense?

In [ ]:
print('CHM minimum value: ', np.nanmin(lidar_chm_xr))
print('CHM max value: ', np.nanmax(lidar_chm_xr))

### Export a Raster to Geotiff Using RioXarray
You can export a raster file in python using the ```rioxarray write()``` function. Export the canopy height model that you just created to your data folder. You will create a new directory called “outputs” within the ```colorado-flood``` directory. This structure allows you to keep things organized, separating your outputs from the data you downloaded.

NOTE: you can use the code below to check for and create an outputs directory. OR, you can create the directory yourself using the finder (MAC) or windows explorer.

In [ ]:
data_path = os.path.join("data","colorado-flood",
                         "spatial",
                         "outputs")

if os.path.exists(data_path):
    print("The directory", data_path, "exists!")
else:
    os.makedirs(data_path)

In [ ]:
# Make sure that your output data has a crs & no data value defined
print("The crs is", lidar_chm_xr.rio.crs)
print("The no data value is", lidar_chm_xr.rio.nodata)

In [ ]:
pre_chm_data_path = os.path.join(data_path, "pre-flood-chm.tif")
pre_chm_data_path

In [ ]:
# Export data to geotiff
lidar_chm_xr.rio.to_raster(pre_chm_data_path)

In [ ]:
# Reopen the data
lidar_chm_data = rxr.open_rasterio(pre_chm_data_path, masked=True).squeeze()
lidar_chm_data

## Manually Reclassify Raster Data
In this lesson, you will learn how to reclassify a raster dataset in Python. When you reclassify a raster, you create a new raster object / file that can be exported and shared with colleagues and / or open in other tools such as QGIS.

In that raster, each pixel is mapped to a new value based on some approach. This approach can vary depending upon your science question.

![_._](img/reclass-raster-esri.gif)

When you reclassify a raster, you create a new raster. In that raster, each cell from the old raster is mapped to the new raster. The values in the new raster are applied using a defined range of values or a raster map. For example above you can see that all cells that contains the values 1-3 are assigned the new value of 5. Source: ESRI
Raster Classification Steps
You can break your raster processing workflow into several steps as follows:

- Data import / cleanup: load and “clean” the data. This includes cropping, removing with nodata values
- Data Exploration: understand the range and distribution of values in your data. This may involve plotting histograms and scatter plots to determine what classes are appropriate for our data
- Reclassify the Data: Once you understand the distribution of your data, you are ready to reclassify. There are statistical and non-statistical approaches to reclassification. Here you will learn how to manually reclassify a raster using bins that you define in your data exploration step.
Please note - working with data is not a linear process. Above you see a potential workflow. You will develop your own workflow and approach.

To get started, first load the required libraries and then open up your raster. In this case, you are using the lidar canopy height model (CHM) that you calculated in the previous lesson.

In [ ]:
# Define relative paths to DTM and DSM files
dtm_path = os.path.join("data","colorado-flood",
                        "spatial",
                        "boulder-leehill-rd",
                        "pre-flood",
                        "lidar",
                        "pre_DTM.tif")

dsm_path = os.path.join("data","colorado-flood",
                        "spatial",
                        "boulder-leehill-rd",
                        "pre-flood",
                        "lidar",
                        "pre_DSM.tif")

# Open DTM and DSM files
pre_lidar_dtm = rxr.open_rasterio(dtm_path, masked=True).squeeze()
pre_lidar_dsm = rxr.open_rasterio(dsm_path, masked=True).squeeze()

# Create canopy height model (CHM)
pre_lidar_chm = pre_lidar_dsm - pre_lidar_dtm
pre_lidar_chm

### What Classification Values to Use?
There are many different approaches to classification. Some use highly sophisticated spatial algorithms that identify patterns in the data that can in turn be used to classify particular pixels into particular “classes”.

In this case, you are simply going to create the classes manually using the range of quantitative values found in our data.

Assuming that our data represent trees (though you know there are likely some buildings in the data), classify your raster into 3 classes:

- Short trees
- Medium trees
- Tall trees

To perform this classification, you need to understand which values represent short trees vs medium trees vs tall trees in your raster. This is where histograms can be extremely useful.

Start by looking at the min and max values in your CHM.

In [ ]:
print('CHM min value:', np.nanmin(pre_lidar_chm))
print('CHM max value:', np.nanmax(pre_lidar_chm))

### Get to Know Raster Summary Statistics
Get to know your data by looking at a histogram. A histogram quantifies the distribution of values found in your data.

In [ ]:
f, ax = plt.subplots()
pre_lidar_chm.plot.hist(color="purple")
ax.set(title="Distribution of Raster Cell Values in the CHM Data",
       xlabel="Height (m)",
       ylabel="Number of Pixels")
plt.show()

### Explore Raster Histograms
Further explore your histogram, by constraining the x axis limits using the xlim and ylim parameters.

These lim parameters visually zooms in on the data in the plot. It does not modify the data.

You might also chose to adjust the number of bins in your plot. Below you plot a bin for each increment on the x axis calculated using:

```hist_range(*xlim)```

You could also set ```bins = 100``` or some other value if you wish.

You can look at the values that Python used to draw your histogram, too.

To do this, you can collect the outputs that are returned when you call ```np.histogram```. This consists of two things:

- ```counts```, which represents the number of items in each bin
- ```bins```, which represents the edges of the bins (there will be one extra item in bins compared to counts)

Each bin represents a bar on your histogram plot. Each bar represents the frequency or number of pixels that have a value within that bin.

Notice that you have adjusted the ```xlim``` and ```ylim``` to zoom into the region of the histogram that you are interested in exploring; however, the values did not actually change.

### Histogram with Custom Breaks
Customize your histogram with breaks that you think might make sense as breaks to use for your raster map.

In the example below, breaks are added in 5 meter increments using the bins = argument.

```bins=[0, 5, 10, 15, 20, 30]```

In [ ]:
# Histogram with custom breaks
f, ax = plt.subplots()
pre_lidar_chm.plot.hist(color="purple",
                        bins=[0, 5, 10, 15, 20, 30])
ax.set(title="Histogram with Custom Breaks",
       xlabel="Height (m)",
       ylabel="Number of Pixels")

plt.show()

You may want to play with the distribution of breaks. Below it appears as if there are many values close to 0.

In the case of this lidar instrument, you know that values between 0 and 2 meters are not reliable (you know this if you read the documentation about the NEON sensor and how these data were processed).

Below you create a bin between 0-2.

You also know you want to create bins for short, medium and tall trees, so experiment with those bins as well.

Below following breaks are used:

- 0 - 2 = no trees
- 2 - 7 = short trees
- 7 - 12 = medium trees
- great than 12 = tall trees

In [ ]:
f, ax = plt.subplots()

pre_lidar_chm.plot.hist(
    color='purple',
    bins=[0, 2, 7, 12, 30])
ax.set(title="Histogram with Custom Breaks",
       xlabel="Height (m)",
       ylabel="Number of Pixels")

plt.show()

You may want to play around with the setting specific bins associated with your science question and the study area. To begin, use the classes above to reclassify your CHM raster.

### Map Raster Values to New Values
To reclassify your raster, first you need to create a reclassification matrix.

This matrix MAPS a range of values to a new defined value. You will use this matrix to create a classified canopy height model where you designate short, medium and tall trees.

The newly defined values will be as follows:

- No trees: (0m - 2m tall) = NA
- Short trees: (2m - 7m tall) = 1
- Medium trees: (7m - 12m tall) = 2
- Tall trees: (> 12m tall) = 3

Notice in the list above that you set cells with a value between 0 and 2 meters to NA or nodata value. This means you are assuming that there are no trees in those locations!

Notice in the matrix below that you use Inf to represent the largest or max value found in the raster. So our assignment is as follows:

- 0 - 2 meters -> 1
- 2 - 7 meters -> 2 (short trees)
- 7 - 12 meters -> 3 (medium trees)
- /> 12 or 12 - Inf -> 4 (tall trees)
Below you create the matrix.

```np.digitize```
Numpy has a function called digitize that is useful for classifying the values in an array. It is similar to how histogram works, because it categorizes datapoints into bins. However, unlike histogram, it doesn’t aggregate/count the number of values within each bin.

Instead, ```digitize``` will replace each datapoint with an integer corresponding to which bin it belongs to. You can use this to determine which datapoints fall within certain ranges.

When you use ```np.digitize```, the bins that you create work as follows:

The starting value by default is included in each bin. The ending value of the bin is not and will be the beginning of the next bin. You can add the argument ```right = True``` if you want the second value in the bin to be included but not the first.

Any values BELOW the bins as defined will be assigned a 0. Any values ABOVE the highest value in your bins will be assigned the next value available. Thus, if you have:

```class_bins = [0, 2, 7, 12, 30]```

Any values that are equal to 30 or larger will be assigned a value of 5. Any values that are < 0 will be assigned a value of 0.

Oftentimes, you can use ```np.inf``` in your array to include all values greater than the last value, and you can use ```-np.inf``` in your array to include all values less than the first value.

However, if you are using the class bins for a BoundaryNorm object for a plot,```np.inf``` will throw an error in matplotlib. The BoundaryNorm object cannot handle an infinity value, so you must supply it with an actual integer.

A good stand in for ```np.inf``` is the maximum value numpy can store as an integer, which can be accessed by using ```np.iinfo(np.int32).max```. This will have the same effect as ```np.inf``` without breaking the BoundaryNorm object.

Likewise, you can use the minimum value of the array ```(arr.min()) ``` instead of ```-np.inf```.

In [ ]:
# Check nodata value for your array
pre_lidar_chm.rio.nodata

Below you define 4 bins. However, you end up with a ```fifth class == 0``` which represents values smaller than 0 which is the minimum value in your chm.

These values < 0 come from the numpy mask fill value which you can see identified above this text.

In [ ]:
data_min_value = np.nanmin(pre_lidar_chm)
data_max_value = np.nanmax(pre_lidar_chm)
print(data_min_value, data_max_value)

In [ ]:
class_bins = [-np.inf, 2, 7, 12, np.inf]
class_bins

In [ ]:
pre_lidar_chm_class = xr.apply_ufunc(np.digitize,
                                     pre_lidar_chm,
                                     class_bins)

In [ ]:
im = pre_lidar_chm_class.plot.imshow()
ax.set_axis_off()

After running the classification, you have one extra class. This class - the first class - is your missing data value. Your classified array output is also a regular (not a masked) array.

You can reassign the first class in your data to a mask using ```xarray .where()```.



In [ ]:
# Mask out values not equalt to 5
pre_lidar_chm_class_ma = pre_lidar_chm_class.where(pre_lidar_chm_class != 5)
pre_lidar_chm_class_ma

In [ ]:
# Plot newly classified and masked raster
f, ax = plt.subplots(figsize=(10,5))
pre_lidar_chm_class_ma.plot.imshow()
ax.set(title="Classified Plot With a Colorbar")

ax.set_axis_off()
plt.show()

In [ ]:
# Plot data using nicer colors
colors = ['linen', 'lightgreen', 'darkgreen', 'maroon']
class_bins = [.5, 1.5, 2.5, 3.5, 4.5]
cmap = ListedColormap(colors)
norm = BoundaryNorm(class_bins, 
                    len(colors))

# Plot newly classified and masked raster
f, ax = plt.subplots(figsize=(10, 5))
pre_lidar_chm_class_ma.plot.imshow(cmap=cmap,
                                   norm=norm)
ax.set(title="Classified Plot With a Colorbar and Custom Colormap (cmap)")
ax.set_axis_off()
plt.show()

### About Spatial Crop
Cropping (sometimes also referred to as clipping), is when you subset or make a dataset smaller, by removing all data outside of the crop area or spatial extent. In this case you have a large raster - but let’s pretend that you only need to work with a smaller subset of the raster.

You can use the crop_image function to remove all of the data outside of your study area. This is useful as it:

Makes the data smaller and
Makes processing and plotting faster
In general when you can, it’s often a good idea to crop your raster data!

To begin let’s load the libraries that you will need in this lesson.

In [ ]:
lidar_chm_path = os.path.join("data","colorado-flood", 
                              "spatial"
                              "boulder-leehill-rd",
                              "outputs",
                              "lidar_chm.tif")

lidar_chm_im = rxr.open_rasterio("data/colorado-flood/spatial/boulder-leehill-rd/outputs/lidar_chm.tif",
                                 masked=True).squeeze()

f, ax = plt.subplots(figsize=(10, 5))
lidar_chm_im.plot.imshow()
ax.set(title="Lidar Canopy Height Model (CHM)")

ax.set_axis_off()
plt.show()

### Open Vector Layer
To begin your clip, open up a vector layer that contains the crop extent that you want to use to crop your data. To open a shapefile you use the ```gpd.read_file()``` function from geopandas. 

In [ ]:
import geopandas as gpd

aoi = os.path.join("data","colorado-flood",
                   "spatial",
                   "boulder-leehill-rd",
                   "clip-extent.shp")

# Open crop extent (your study area extent boundary)
crop_extent = gpd.read_file(aoi)


Next, view the coordinate reference system (CRS) of both of your datasets. Remember that in order to perform any analysis with these two datasets together, they will need to be in the same CRS.



In [ ]:
print('crop extent crs: ', crop_extent.crs)
print('lidar crs: ', lidar_chm_im.rio.crs)

In [ ]:
# Plot the crop boundary layer
# Note this is just an example so you can see what it looks like
# You don't need to plot this layer in your homework!
fig, ax = plt.subplots(figsize=(6, 6))

crop_extent.plot(ax=ax)

ax.set_title("Shapefile Crop Extent",
             fontsize=16)
plt.show()

### Clip Raster Data Using RioXarray .clip
If you want to crop the data you can use the ```rio.clip``` function. When you clip the data, you can then export it and share it with colleagues. Or use it in another analysis.

To perform the clip you:

1. Open the raster dataset that you wish to crop using xarray or rioxarray.
2. Open your shapefile as a geopandas object.
3. Crop the data using the ```.clip()``` function.
.clip has several parameters that you can consider including

- ```drop = True``` : The default. setting it will drop all pixels outside of the clip extent
- ```invert = False``` : The default. If set to true it will clip all data INSIDE of the clip extent
- ```crs``` : if your shapefile is in a different CRS than the raster data, pass the CRS to ensure the data are clipped correctly.
Below you clip the data to the extent of the AOI geodataframe imported above. The data are then plotted.

In [ ]:
lidar_clipped = lidar_chm_im.rio.clip(crop_extent.geometry.apply(mapping),
                                      # This is needed if your GDF is in a diff CRS than the raster data
                                      crop_extent.crs)

f, ax = plt.subplots(figsize=(10, 4))
lidar_clipped.plot(ax=ax)
ax.set(title="Raster Layer Cropped to Geodataframe Extent")
ax.set_axis_off()
plt.show()

In [ ]:
path_to_tif_file = os.path.join("data","colorado-flood",
                                "spatial",
                                "outputs",
                                "lidar_chm_cropped.tif")

# Write the data to a new geotiff file
lidar_clipped.rio.to_raster(path_to_tif_file)

In [ ]:
# Open the data you wrote out above
clipped_chm = rxr.open_rasterio(path_to_tif_file)

# Customize your plot as you wish!
f, ax = plt.subplots(figsize=(10, 4))
clipped_chm.plot(ax=ax,
                 cmap='Greys')
ax.set(title="Final Clipped CHM")
ax.set_axis_off()
plt.show()